In [164]:
import pymysql.cursors
from dotenv import load_dotenv
import os

import pandas as pd
import numpy as np

load_dotenv()

host = os.getenv("db_host")
user = os.getenv("db_user")
password = os.getenv("db_password")
port = int(os.getenv("db_port"))
database = os.getenv("db_database")

In [165]:
connection = pymysql.connect(host=host,
                             user=user,
                             password=password,
                             database=database,
                             port=port,
                             cursorclass=pymysql.cursors.DictCursor)

In [166]:
# 연결 테스트

with connection:
    with connection.cursor() as cursor:
        sql = "select * from tb_slalteloghis where SERIALNO=%s"
        cursor.execute(sql, ('3030303030333538313232303835303133393434'))
        result = cursor.fetchone()
        print(result)

{'UNTID': '4098610941', 'IVTID': 'IVT-2023112000001', 'SERIALNO': '3030303030333538313232303835303133393434', 'EVTDATME': datetime.datetime(2024, 1, 1, 6, 0, 13), 'READTIME': '20240101 0600', 'TYPECODE': 1, 'DATAREAD': 20, 'DATATYPE': 1, 'PHASE': 2, 'STATIONNO': 1, 'ERRCD': 57, 'INV1': 0.0, 'INV2': 0.0, 'INV3': 0.0, 'INV4': 0.0, 'INV5': 0.0, 'INV6': 0.0, 'INV7': 0.0, 'INV8': 0.0, 'INV9': 0.0, 'INV10': 0.0, 'INV11': 0.0, 'INV12': 0.0, 'INV13': 0.0, 'INV14': 0.0, 'INV15': 0.0, 'INV16': 0.0, 'INV17': 0.0, 'INV18': 0.0, 'INV19': 0.0, 'INV20': 0.0, 'INA1': 0.0, 'INA2': 0.0, 'INA3': 0.0, 'INA4': 0.0, 'INA5': 0.0, 'INA6': 0.0, 'INA7': 0.0, 'INA8': 0.0, 'INA9': 0.0, 'INA10': 0.0, 'INA11': 0.0, 'INA12': 0.0, 'INA13': 0.0, 'INA14': 0.0, 'INA15': 0.0, 'INA16': 0.0, 'INA17': 0.0, 'INA18': 0.0, 'INA19': 0.0, 'INA20': 0.0, 'INP': 0.0, 'OUTV': 0.0, 'OUTA': 0.0, 'OUTVR': 0.0, 'OURVS': 0.0, 'OUTVT': 0.0, 'OUTAR': 0.0, 'OUTAS': 0.0, 'OUTAT': 0.0, 'OUTP': 0.0, 'PF': 0.0, 'FRQ': 0.0, 'TEMP': 0.0, 'TPG': D

In [167]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

table_type = {'lte':'tb_slalteloghis', 'lora':'tb_slaloraloghis', 'gsrems':'tb_slagsremsloghis'}

def generate_monthly_first_date_list(type, start_date, end_date):
    table_list = []
    current_date = start_date.replace(day=1)  # 시작 날짜의 1일로 설정

    while current_date <= end_date:
        table_list.append(f"{table_type[type]}_{current_date.strftime('%Y_%m_%d')}")
        current_date += relativedelta(months=1)  # 매월 1일로 이동

    table_list.append(f"{table_type[type]}")

    return table_list

# 입력된 날짜
input_date = datetime(2023, 7, 11)

# 현재 날짜
current_date = datetime.now()

# 모뎀 타입
type = 'lte'

# 테이블 리스트 생성
table_list = generate_monthly_first_date_list(type, input_date, current_date)

# 결과 출력
print(table_list)

['tb_slalteloghis_2023_07_01', 'tb_slalteloghis_2023_08_01', 'tb_slalteloghis_2023_09_01', 'tb_slalteloghis_2023_10_01', 'tb_slalteloghis_2023_11_01', 'tb_slalteloghis_2023_12_01', 'tb_slalteloghis_2024_01_01', 'tb_slalteloghis']


In [168]:
SERIALNO = "3030303030333538313232303835303133393434"

ascii_serialno = "00000358122085013944"
hex_serialno = ''.join(hex(ord(char))[2:] for char in ascii_serialno)

print(hex_serialno)

3030303030333538313232303835303133393434


In [169]:
sql = " UNION ".join(f"SELECT * FROM {table} WHERE SERIALNO = '{hex_serialno}'" for table in table_list)

print(sql)

SELECT * FROM tb_slalteloghis_2023_07_01 WHERE SERIALNO = '3030303030333538313232303835303133393434' UNION SELECT * FROM tb_slalteloghis_2023_08_01 WHERE SERIALNO = '3030303030333538313232303835303133393434' UNION SELECT * FROM tb_slalteloghis_2023_09_01 WHERE SERIALNO = '3030303030333538313232303835303133393434' UNION SELECT * FROM tb_slalteloghis_2023_10_01 WHERE SERIALNO = '3030303030333538313232303835303133393434' UNION SELECT * FROM tb_slalteloghis_2023_11_01 WHERE SERIALNO = '3030303030333538313232303835303133393434' UNION SELECT * FROM tb_slalteloghis_2023_12_01 WHERE SERIALNO = '3030303030333538313232303835303133393434' UNION SELECT * FROM tb_slalteloghis_2024_01_01 WHERE SERIALNO = '3030303030333538313232303835303133393434' UNION SELECT * FROM tb_slalteloghis WHERE SERIALNO = '3030303030333538313232303835303133393434'


In [170]:
connection = pymysql.connect(host=host,
                             user=user,
                             password=password,
                             database=database,
                             port=port,
                             cursorclass=pymysql.cursors.DictCursor)

In [171]:
try :
    with connection:
        with connection.cursor() as cursor:
            cursor.execute(sql)
            result = cursor.fetchall()

            df = pd.DataFrame(result)

            csv_filename = 'solar_data.csv'
            df.to_csv(csv_filename, index=False, encoding='utf-8')

            print(f"Query result has been saved to {csv_filename}")
except:
    connection.close()
    print(f"Query Faild...")

Query result has been saved to solar_data.csv
